In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc numpy qiskit-addon-cutting

*Орієнтовний час виконання: вісім хвилин на процесорі Eagle (ПРИМІТКА: Це лише орієнтовна оцінка. Фактичний час виконання може відрізнятися.)*

## Передумови

Цей посібник демонструє, як побудувати `Qiskit pattern` для розрізання вентилів у квантовій схемі з метою зменшення глибини схеми. Для більш детального обговорення розрізання схем відвідайте [документацію додатка Qiskit для розрізання схем](https://qiskit.github.io/qiskit-addon-cutting/).

## Вимоги

Перед початком роботи з цим посібником переконайся, що у тебе встановлено наступне:
- Qiskit SDK v2.0 або новішої версії з підтримкою [візуалізації](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.22 або новішої версії (`pip install qiskit-ibm-runtime`)
- Додаток Qiskit для розрізання схем v0.9.0 або новішої версії (`pip install qiskit-addon-cutting`)

## Налаштування

In [1]:
import numpy as np

from qiskit.circuit.library import EfficientSU2
from qiskit.quantum_info import PauliList, Statevector, SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

from qiskit_addon_cutting import (
    cut_gates,
    generate_cutting_experiments,
    reconstruct_expectation_values,
)

from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

## Крок 1: Відображення класичних вхідних даних на квантову задачу
Ми реалізуємо наш шаблон Qiskit, використовуючи чотири кроки, описані в [документації](/guides/intro-to-patterns). У цьому випадку ми будемо симулювати очікувані значення на схемі певної глибини шляхом розрізання вентилів, що призводить до swap-вентилів, та виконання підексперименів на схемах меншої глибини. Розрізання вентилів є актуальним для кроків 2 (оптимізація схеми для квантового виконання шляхом декомпозиції віддалених вентилів) та 4 (постобробка для реконструкції очікуваних значень на оригінальній схемі).
На першому кроці ми згенеруємо схему з бібліотеки схем Qiskit та визначимо деякі спостережувані величини.

*   Вхідні дані: Класичні параметри для визначення схеми
*   Вихідні дані: Абстрактна схема та спостережувані величини

In [2]:
circuit = EfficientSU2(num_qubits=4, entanglement="circular").decompose()
circuit.assign_parameters([0.4] * len(circuit.parameters), inplace=True)
observables = PauliList(["ZZII", "IZZI", "IIZZ", "XIXI", "ZIZZ", "IXIX"])
circuit.draw("mpl", scale=0.8, style="iqp")

<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/54ed0f13-0.avif" alt="Output of the previous code cell" />

![Результат виконання попередньої комірки коду](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/54ed0f13-0.avif)

## Крок 2: Оптимізація задачі для виконання на квантовому обладнанні
*   Вхідні дані: Абстрактна схема та спостережувані величини
*   Вихідні дані: Цільова схема та спостережувані величини, отримані шляхом розрізання віддалених вентилів для зменшення глибини транспільованої схеми

Ми обираємо початкову розкладку, яка вимагає двох swap-операцій для виконання вентилів між кубітами 3 та 0, і ще двох swap-операцій для повернення кубітів у їхні початкові позиції. Ми обираємо `optimization_level=3`, що є найвищим рівнем оптимізації, доступним у попередньо налаштованому менеджері проходів.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, min_num_qubits=circuit.num_qubits, simulator=False
)

pm = generate_preset_pass_manager(
    optimization_level=3, initial_layout=[0, 1, 2, 3], backend=backend
)
transpiled_qc = pm.run(circuit)

![Карта зв'язності, що показує кубіти, які потребуватимуть swap-операцій](../docs/images/tutorials/depth-reduction-with-circuit-cutting/swaps.avif)

In [4]:
print(f"Transpiled circuit depth: {transpiled_qc.depth()}")
transpiled_qc.draw("mpl", scale=0.4, idle_wires=False, style="iqp", fold=-1)

Transpiled circuit depth: 103


<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/4fe4af43-1.avif" alt="Output of the previous code cell" />

*Find and cut the distant gates:* We will replace the distant gates (gates connecting non-local qubits, 0 and 3) with `TwoQubitQPDGate` objects by specifying their indices. `cut_gates` will replace the gates in the specified indices with `TwoQubitQPDGate` objects and also return a list of `QPDBasis` instances -- one for each gate decomposition. The `QPDBasis` object contains information about how to decompose the cut gates into single-qubit operations.

In [5]:
# Find the indices of the distant gates
cut_indices = [
    i
    for i, instruction in enumerate(circuit.data)
    if {circuit.find_bit(q)[0] for q in instruction.qubits} == {0, 3}
]

# Decompose distant CNOTs into TwoQubitQPDGate instances
qpd_circuit, bases = cut_gates(circuit, cut_indices)

qpd_circuit.draw("mpl", scale=0.8)

<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/23e3d25e-0.avif" alt="Output of the previous code cell" />

![Результат виконання попередньої комірки коду](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/4fe4af43-1.avif)

*Знаходження та розрізання віддалених вентилів:* Ми замінимо віддалені вентилі (вентилі, що з'єднують нелокальні кубіти, 0 та 3) об'єктами `TwoQubitQPDGate`, вказавши їхні індекси. `cut_gates` замінить вентилі за вказаними індексами об'єктами `TwoQubitQPDGate`, а також поверне список екземплярів `QPDBasis` — по одному для кожної декомпозиції вентиля. Об'єкт `QPDBasis` містить інформацію про те, як декомпозувати розрізані вентилі на однокубітні операції.

In [6]:
# Generate the subexperiments and sampling coefficients
subexperiments, coefficients = generate_cutting_experiments(
    circuits=qpd_circuit, observables=observables, num_samples=np.inf
)

![Результат виконання попередньої комірки коду](../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/23e3d25e-0.avif)

*Генерація підексперименів для виконання на бекенді*: `generate_cutting_experiments` приймає схему, що містить екземпляри `TwoQubitQPDGate`, та спостережувані величини у вигляді `PauliList`.

Для симуляції очікуваного значення повнорозмірної схеми з квазіймовірнісного розподілу декомпозованих вентилів генерується багато підексперименів, які потім виконуються на одному або кількох бекендах. Кількість зразків, взятих з розподілу, контролюється параметром `num_samples`, і для кожного унікального зразка надається один комбінований коефіцієнт. Для отримання додаткової інформації про обчислення коефіцієнтів зверніться до [пояснювальних матеріалів](https://qiskit.github.io/qiskit-addon-cutting/explanation/index.html).

In [7]:
# Transpile the decomposed circuit to the same layout
transpiled_qpd_circuit = pm.run(subexperiments[100])

print(f"Original circuit depth after transpile: {transpiled_qc.depth()}")
print(
    f"QPD subexperiment depth after transpile: {transpiled_qpd_circuit.depth()}"
)
transpiled_qpd_circuit.draw(
    "mpl", scale=0.6, style="iqp", idle_wires=False, fold=-1
)

Original circuit depth after transpile: 103
QPD subexperiment depth after transpile: 46


<Image src="../docs/images/tutorials/depth-reduction-with-circuit-cutting/extracted-outputs/70e2f1b6-1.avif" alt="Output of the previous code cell" />

*Для порівняння: підексперименти QPD будуть мати меншу глибину після розрізання віддалених вентилів*: Ось приклад довільно обраного підексперименту, згенерованого зі схеми QPD. Його глибина була зменшена більш ніж удвічі. Необхідно згенерувати та обчислити багато таких імовірнісних підексперименів для реконструкції очікуваного значення більш глибокої схеми.

In [8]:
print(f"Sampling overhead: {np.prod([basis.overhead for basis in bases])}")

Sampling overhead: 729.0


## Step 3: Execute using Qiskit primitives

Execute the target circuits ("subexperiments") with the Sampler Primitive.

*   Input: Target circuits
*   Output: Quasi-probability distributions

In [9]:
# Transpile the subexperiments to the backend's instruction set architecture (ISA)
isa_subexperiments = pm.run(subexperiments)

# Set up the Qiskit Runtime Sampler primitive.  For a fake backend, this will use a local simulator.
sampler = SamplerV2(backend)

# Submit the subexperiments
job = sampler.run(isa_subexperiments)

In [11]:
# Retrieve the results
results = job.result()

In [10]:
print(job.job_id())

czypg1r6rr3g008mgp6g


## Крок 3: Виконання за допомогою примітивів Qiskit
Виконайте цільові схеми ("підексперименти") за допомогою примітива Sampler.

*   Вхідні дані: Цільові схеми
*   Вихідні дані: Квазіймовірнісні розподіли

In [12]:
reconstructed_expvals = reconstruct_expectation_values(
    results,
    coefficients,
    observables,
)
# Reconstruct final expectation value
final_expval = np.dot(reconstructed_expvals, [1] * len(observables))
print("Final reconstructed expectation value")
print(final_expval)

Final reconstructed expectation value
1.0751342773437473


In [13]:
ideal_expvals = [
    Statevector(circuit).expectation_value(SparsePauliOp(observable))
    for observable in observables
]
print("Ideal expectation value")
print(np.dot(ideal_expvals, [1] * len(observables)).real)

Ideal expectation value
1.2283177520039992


## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_2ftYFf9t72yFNIO)